In [4]:
import pandas as pd
import logging
import numpy as np
import re
import string
import torch.nn as nn
import unidecode

# from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import (
    XLNetTokenizer,
    XLNetModel,
    XLNetForSequenceClassification,
    AutoModelForSequenceClassification)
    # TrainingArguments,
    # Trainer)
# import evaluate

In [5]:
def preprocess_text(text_arr):
    output_text = []
    for x in text_arr:
        processed = str(unidecode.unidecode(re.sub('[%s]' % re.escape(string.punctuation), '' , x))).lower()
        output_text.append(processed)
        
    return output_text

In [6]:
fulltrain = pd.read_csv('./fulltrain.csv', names = ['label', 'text'])

full_data = pd.DataFrame(preprocess_text(fulltrain['text']))
full_data['label'] = fulltrain['label'] - 1 # make the labels 0-indexed

# split dataset into train and validation
# X_train, X_test, y_train, y_test = train_test_split(
#     texts, fulltrain['label'], test_size=0.2, shuffle=True, stratify=fulltrain['label'], random_state=42
# )
# print("len train set = {}, len test set = {}".format(len(X_train), len(X_test)))

# train_df = pd.DataFrame(X_train)
# train_df['target'] = y_train
# test_df = pd.DataFrame(X_test)
# test_df['target'] = y_test
# class_names = [0, 1, 2, 3]
# train_df.shape, test_df.shape

In [7]:
train, test = train_test_split(full_data, test_size=0.2, shuffle=True, random_state=42)

# data = DatasetDict()
# data['train'] = Dataset.from_pandas(train)
# data['test'] = Dataset.from_pandas(test)

In [5]:
# class XLNet_Model(nn.Module):
#   def __init__(self, classes):
#     super(XLNet_Model, self).__init__()
#     self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased')
#     self.out = nn.Linear(self.xlnet.config.hidden_size, classes)
#   def forward(self, input):
#     outputs = self.xlnet(**input)
#     out = self.out(outputs.last_hidden_state)
#     return out

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# data_tok = data.map(
#     lambda x: tokenizer(x['text'], padding="max_length", truncation=True),
#     batched=True,
#     batch_size=128
# )
# data_tok['train'][0]

100%|████████████████████████████████| 798011/798011 [00:01<00:00, 486934.95B/s]


In [11]:
# train_data = data_tok["train"].shuffle(seed=123)
# test_data = data_tok["test"].shuffle(seed=123)
!pip list torch

Package                 Version
----------------------- -----------
absl-py                 1.4.0
aiohttp                 3.8.4
aiosignal               1.3.1
altair                  4.2.2
anyio                   3.5.0
appdirs                 1.4.4
appnope                 0.1.2
argon2-cffi             21.3.0
argon2-cffi-bindings    21.2.0
asttokens               2.0.5
async-timeout           4.0.2
attrs                   22.2.0
Babel                   2.11.0
backcall                0.2.0
beautifulsoup4          4.11.1
bleach                  4.1.0
blinker                 1.5
boto3                   1.24.28
botocore                1.27.59
Bottleneck              1.3.5
brotlipy                0.7.0
cachetools              5.3.0
certifi                 2022.12.7
cffi                    1.15.1
chardet                 4.0.0
charset-normalizer      2.0.4
click                   8.1.3
comm                    0.1.2
cryptography            39.0.1
datasets                2.10.1
debugpy           

In [10]:
from transformers import TrainingArguments, Trainer

ImportError: cannot import name 'TrainingArguments' from 'transformers' (/Users/sagarsureka/opt/anaconda3/envs/xlnet-conda-env/lib/python3.11/site-packages/transformers/__init__.py)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=4)
training_args = TrainingArguments(output_dir="checkpoints", evaluation_strategy="epoch")
metric = evaluate.load("accuracy")

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,
)